In [6]:
_predict_df = "select (Version.[Version Name]*Product.[Product].[196426]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource,PluginSetting

# register inputs
predict_df = O9DataLake.register("predict_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _predict_df,plugin_setting = PluginSetting.Inputs,spark = spark)
systemreport = O9DataLake.register("sysreport",data_source = DataSource.LIVEFRAME,entity_type = ResourceType.LIVEFRAME,plugin_setting = PluginSetting.Inputs,spark = spark)

# register slice dimension
O9DataLake.register("Product.[Product]", data_source = DataSource.LS, entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.SliceDimension)

# register outputs
O9DataLake.register("output1",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)

# register script params
script_params = O9DataLake.register({"var1":"10","var2":"Test1"}, data_source = DataSource.LS, plugin_setting = PluginSetting.ScriptParam)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-02 07:43:04,208 - o9_logger - ERROR - Output variable already exists.

In [7]:
O9DataLake.inputs

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'predict_df': {'name': 'predict_df', 'resource_type': <ResourceType.IBPL: 'ibpl_query'>, 'data_source': <DataSource.LS: 'liveserver'>, 'query': 'select (Version.[Version Name]*Product.[Product].[196426]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});', 'std_count_limit': '200000', 'df': DataFrame[VersionVersionName: string, ProductProduct: string, TimeFiscalWeek: string, SalesAccountAccount: string, LocationLocation: string, DPSellOutUnitsActuals: float, MeanPricingSavePCT: float, PlacementCount: float, PromotionCount: float, DPSellOutPrice: float]}, 'sysreport': {'name': 'sysreport', 'resource_type': <ResourceType.LIVEFRAME: 'liveframe'>, 'data_source': <DataSource.LIVEFRAME: 'liveframe'>, 'query': 'SELECT ([WalmartTime].[Day].filter(#.[Day$InActive] in { false }) * [Store].[Store_ID].filter(#.[Store_ID$InActive] in { false }) * [

In [8]:
type(systemreport)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [9]:
systemreport.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------+------------------+-----------+
|WalmartTimeDay|StoreStoreID|VersionVersionName|DSMLdsmldim|
+--------------+------------+------------------+-----------+
|    02-05-2010|           1|CurrentWorkingView|      2.572|
|    02-05-2010|          10|CurrentWorkingView|      3.963|
|    02-05-2010|          11|CurrentWorkingView|      2.653|
|    02-05-2010|          12|CurrentWorkingView|      4.121|
|    02-05-2010|          13|CurrentWorkingView|      2.797|
|    02-05-2010|          14|CurrentWorkingView|      3.951|
|    02-05-2010|          15|CurrentWorkingView|      2.913|
|    02-05-2010|          16|CurrentWorkingView|      3.764|
|    02-05-2010|          17|CurrentWorkingView|      2.875|
|    02-05-2010|          18|CurrentWorkingView|      3.577|
|    02-05-2010|          19|CurrentWorkingView|      2.999|
|    02-05-2010|           2|CurrentWorkingView|      3.308|
|    02-05-2010|          20|CurrentWorkingView|      3.647|
|    02-05-2010|        

In [10]:
# fetching inputs
predict_df = O9DataLake.get('predict_df')
systemreport = O9DataLake.get('sysreport')

# fetching script params
value1 = O9DataLake.get_script_param("var1")
value2 = O9DataLake.get_script_param("var2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#user script

# package imports
import logging
from sklearn import tree

# initialize output variables
output1 = None


# initialize logger
logger = logging.getLogger('o9_logger')

logger.debug(f'predict_df dataframe:  {predict_df.count()}')
logger.debug(f'systemreport dataframe:  {systemreport.count()}')
logger.debug(f'script param var1 value: {value1}')
logger.debug(f'script param var2 value: {value2}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-02 07:47:31,476 - o9_logger - DEBUG - predict_df dataframe:  14
2024-05-02 07:47:31,736 - o9_logger - DEBUG - systemreport dataframe:  8190
2024-05-02 07:47:31,736 - o9_logger - DEBUG - script param var1 value: 10
2024-05-02 07:47:31,736 - o9_logger - DEBUG - script param var2 value: Test1

In [14]:
output1 = systemreport
logger.debug(f'systemreport dataframe:  {systemreport.head()}')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-02 07:47:42,115 - o9_logger - DEBUG - systemreport dataframe:  Row(WalmartTimeDay='02-05-2010', StoreStoreID='1', VersionVersionName='CurrentWorkingView', DSMLdsmldim=2.572000026702881)

In [15]:
#pushing outputs to Live Server(make sure all dimensions,measures present in LiveServer)
O9DataLake.put('output1', output1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-02 07:47:59,001 - o9_logger - DEBUG - HDFS enabled: False
Failed to save output to Live Server
[Errno 13] Permission denied: '/mnt/resource'